In [7]:
import time
import numpy as np
import pandas as pd
from amuse.lab import Particles
from amuse.couple import bridge
import matplotlib.pyplot as plt
from amuse.units.constants import G
from amuse.units import units, constants
from amuse.lab import Huayno, nbody_system
from amuse.community.ph4.interface import ph4
from amuse.ext.orbital_elements import get_orbital_elements_from_binary
from amuse.ext.orbital_elements import new_binary_from_orbital_elements

In [2]:
test_particles = Particles(3)
test_particles[0].position = [0,0,0] | units.m
test_particles[0].velocity = [0,0,0] | units.m/units.s
test_particles[1].position = [1,0,0] | units.m
test_particles[1].velocity = [0,2,0] | units.m/units.s
test_particles[2].position = [0,3,0] | units.m
test_particles[2].velocity = [-1,0,0] | units.m/units.s
for i in range(3):
    test_particles[i].mass = 1 |units.kg 
    test_particles[i].name = str(i)
    
E_kin = test_particles.kinetic_energy()
E_pot = test_particles.potential_energy()

test_particles.mass.value_in(units.kg)
E_kin

test_particles[0].radius = 1 | units.m 
test_particles[1].radius = 2 | units.m
test_particles[2].radius = 3 | units.m 

print(test_particles.radius)
print(test_particles.position)
print(test_particles.position/test_particles.radius.reshape((3,1)))

[1.0, 2.0, 3.0] m
[[0.0, 0.0, 0.0], [1.0, 0.0, 0.0], [0.0, 3.0, 0.0]] m
[[ 0.   0.   0. ]
 [ 0.5  0.   0. ]
 [ 0.   1.   0. ]]


AttributeError: module 'amuse.couple.bridge' has no attribute 'info'

In [4]:
print(test_particles[test_particles.name!=str(1)])

                 key         mass         name           vx           vy           vz            x            y            z
                   -           kg         none        m / s        m / s        m / s            m            m            m
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
 5729052259097987684    1.000e+00            0    0.000e+00    0.000e+00    0.000e+00    0.000e+00    0.000e+00    0.000e+00
 6903053455073453869    1.000e+00            2   -1.000e+00    0.000e+00    0.000e+00    0.000e+00    3.000e+00    0.000e+00
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========


AttributeError: You tried to access attribute 'add_attribute' but this attribute is not defined for this set.

In [5]:
a = [0,1,2,3]
b = [2,3,4,5]
c = [2,5,6,2]

df = pd.DataFrame({'a':a,
                   'b':b,
                   'c':c})
df

,a,b,c
0,0,2,2
1,1,3,5
2,2,4,6
3,3,5,2


In [10]:
end = 10|units.yr
dt = 1|units.yr
times = np.arange(0,end.value_in(units.yr),dt.value_in(units.yr))*1|units.yr
times

quantity<[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] yr>

In [31]:
def energy_dissipation(system,k=0.565,alpha=0.1):
    """ Calculates the total energy dissipation due to the tidal force for a particle system. The second
    particle in the particle set is the particle that exerts the tidal force."""
    #defining some constants
    R = (1|units.RJupiter).value_in(units.m) #radius of Jupiter
    M = (1|units.MJupiter).value_in(units.kg)
    G = constants.G.value_in( (units.m)**3 / (units.kg * units.s**2) )
    
    #getting some parameters/values from the particle set
    m = system.mass.value_in(units.kg)
    r = np.sqrt(np.sum(np.square(system.position.value_in(units.m) - \
                                 system[1].position.value_in(units.m)), axis=1))
    
    #to exclude the force of Jupiter on itself
    mask = np.arange(len(system)) != 1
    
    #the energy dissipation
    dE_dt = -9*k*G**(3/2)*M**(1/2)*m[mask]**2*R**5*np.sin(2*alpha)/(2*r[mask]**(15/2))
    
    return np.sum(dE_dt) * 1|units.J/units.s

def get_energies(system,timestep,k=0.565,alpha=0.1,):
    """ Calculates the total energy of the system, along with the energy dissipation."""
    #getting the kinetic and potential energy from the particle system
    E_kin = system.kinetic_energy()
    E_pot = system.potential_energy()
    E_tot = E_kin + E_pot
    
    #getting the energy dissipation
    dE_dt = energy_dissipation(system)
    #getting the total dissipated energy in the following timestep
    dE = dE_dt*timestep
    
    return E_tot, dE

#get_energies(test_particles,10|units.s)
help(energy_dissipation)

Help on function energy_dissipation in module __main__:

energy_dissipation(system, k=0.565, alpha=0.1)
    Calculates the total energy dissipation due to the tidal force for a particle system. The second
    particle in the particle set is the particle that exerts the tidal force.



In [ ]:
# Tidal forces
class TidalForce():
    def __init__(self, alpha=0.1, method='simple'):
        self.alpha=alpha
        self.method=method
        
    
    def add_particles(self, particle_set):
        self.particles = particle_set
    
    
    def tidal_acceleration(self, k, alpha):
        R = (1|units.RJupiter).value_in(units.m) 
        M = (1|units.MJupiter).value_in(units.kg)
        G = constants.G.value_in( (units.m)**3 / (units.kg * units.s**2) )
        r = np.sqrt(np.sum(np.square(self.particles.position.value_in(units.m) - \
                                     self.particles[1].position.value_in(units.m)), axis=1))
        
        abs_vel = np.sqrt(np.sum(np.square(self.particles.velocity.value_in(units.m*units.s**-1)), axis=1))
        abs_acc = np.zeros(len(self.particles))
        mask = np.arange(len(self.particles)) != 1
        abs_acc[mask] = -9*k*G*self.particles.mass[mask].value_in(units.kg)*(R**5)*\
        np.sin(2*alpha)/(4*r[mask]**7)
        abs_acc = abs_acc * 1|units.m*units.s**-2
                
        ax = abs_acc*(self.particles.vx.value_in(units.m*units.s**-1)/abs_vel)
        ay = abs_acc*(self.particles.vy.value_in(units.m*units.s**-1)/abs_vel)
        az = abs_acc*(self.particles.vz.value_in(units.m*units.s**-1)/abs_vel)
        
        return ax, ay, az
    
    
    def get_gravity_at_point(self, eps, x, y, z):
        if self.method == 'simple':
            return self.tidal_acceleration(k=0.565, alpha=self.alpha) #k from literature, alpha guess
        elif self.method == 'complex':
            return self.comp_tidal_acceleration(kdt=2e-2) #k from literature, alpha guess
    
    def comp_tidal_acceleration(self, kdt):
        #the number of satellites is needed to reshape the arrays such that all arrays can be contracted
        #in the correct way (eg multiplication of arrays of vectors, with arrays)
        num_sat = len(self.particles)-1
        mask = self.particles.name != 'jupiter'
        
        #initializing some constants
        M = (1|units.MJupiter).value_in(units.kg)
        G = constants.G.value_in( (units.m)**3 / (units.kg * units.s**2) )
        c = constants.c.value_in( units.m/units.s)
        
        #obtaining some attributes of the sattelites
        m = (self.particles.mass.value_in(units.kg)[mask]).reshape((num_sat,1))
        R_sat = (self.particles.radius.value_in(units.m)[mask]).reshape((num_sat,1))
        
        #calculating the relative positions/distances and velocities 
        r_vec = self.particles[mask].position.value_in(units.m) - self.particles[1].position.value_in(units.m)
        r = np.sqrt(np.sum(np.square(r_vec),axis=1)).reshape((num_sat,1))
        
        v_vec = self.particles[mask].velocity.value_in(units.m/units.s) - \
                self.particles[1].velocity.value_in(units.m/units.s)
        v = np.sqrt(np.sum(np.square(v_vec),axis=1)).reshape((num_sat,1))
        
        omega = v_vec/r
        
        #calculating the tidal force
        f = -3*kdt*G*(M**2)*(R_sat**5)/(r**10)*( 2*r_vec*(np.sum(r_vec*v_vec, axis=1).reshape((num_sat,1))) +\
                                                 (r**2)*( np.cross(r_vec,omega) + v_vec ))
        
        #calculating the general relativity correction
        f_rel = G*m*M/(c**2 * (r)**3)*( (4*G*M/r - v**2)*r_vec + \
                        4*(np.sum(r_vec*v_vec,axis=1).reshape(num_sat,1))*v_vec)
        
        #calculating the acceleration due to tidal effects
        a = (M + m)/(M*m)*(f+f_rel)
        
        ax = np.zeros(num_sat+1)
        ay = np.zeros(num_sat+1)
        az = np.zeros(num_sat+1)
        
        ax[mask] = a[:,0]
        ay[mask] = a[:,1]
        az[mask] = a[:,2]
        
        ax = ax * 1 | units.m/(units.s**2)
        ay = ay * 1 | units.m/(units.s**2)
        az = az * 1 | units.m/(units.s**2)
        
        return ax, ay, az
    

def energy_dissipation(system, k=0.565, alpha=0.1):
    """ Calculates the total energy dissipation due to the tidal force for a particle system. The 
        second particle in the particle set is the particle that exerts the tidal force. """

    # Defining some constants
    R = (1|units.RJupiter).value_in(units.m) #radius of Jupiter
    M = (1|units.MJupiter).value_in(units.kg)
    G = constants.G.value_in( (units.m)**3 / (units.kg * units.s**2) )
    
    # Getting some parameters/values from the particle set
    m = system.mass.value_in(units.kg)
    r = np.sqrt(np.sum(np.square(system.position.value_in(units.m) - \
                                 system[1].position.value_in(units.m)), axis=1))
    
    # To exclude the force of Jupiter on itself
    mask = np.arange(len(system)) != 1
    
    # The energy dissipation
    dE_dt = -9*k*G**(3/2)*M**(1/2)*m[mask]**2*R**5*np.sin(2*alpha)/(2*r[mask]**(15/2))
   
    return np.sum(dE_dt) * 1|units.J/units.s


def get_energies(system, timestep, k=0.565, alpha=0.1):
    """ Calculates the total energy of the system, along with the energy dissipation. """

    # Getting the kinetic and potential energy from the particle system
    E_kin = system.kinetic_energy()
    E_pot = system.potential_energy()
    E_tot = E_kin + E_pot
    
    # Getting the energy dissipation
    dE_dt = energy_dissipation(system)

    # Getting the total dissipated energy in the following timestep
    dE = dE_dt*timestep
    
    return E_tot, dE